## Importando as Bibliotecas

In [39]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

## Carregar e Filtrar os Dados de 5 Anos

In [15]:
df = pd.read_csv("C:/Users/Cristiano/OneDrive - Belago Technologies/Documentos/projeto_fnt/data/base_completa_araraquara.csv")
df.head(10)

,data,regiao,operacao,original,temperatura_maxima,temperatura_minima,status
0,2023-11-08,NORTE,ZSOZAR-ZARZOI3-244450,thermo_id_20001336,54.2,24.7,Normal
1,2023-11-09,NORTE,ZSOZAR-ZARZOI3-244450,thermo_id_20001336,49.1,19.1,Normal
2,2023-11-10,NORTE,ZSOZAR-ZARZOI3-244450,thermo_id_20001336,56.1,19.6,Normal
3,2023-11-11,NORTE,ZSOZAR-ZARZOI3-244450,thermo_id_20001336,56.4,18.8,Normal
4,2023-11-12,NORTE,ZSOZAR-ZARZOI3-244450,thermo_id_20001336,57.6,20.6,Normal
5,2023-11-13,NORTE,ZSOZAR-ZARZOI3-244450,thermo_id_20001336,55.9,20.2,Normal
6,2023-11-14,NORTE,ZSOZAR-ZARZOI3-244450,thermo_id_20001336,54.8,21.0,Normal
7,2023-11-15,NORTE,ZSOZAR-ZARZOI3-244450,thermo_id_20001336,57.3,18.8,Normal
8,2023-11-16,NORTE,ZSOZAR-ZARZOI3-244450,thermo_id_20001336,52.1,18.8,Normal
9,2023-11-17,NORTE,ZSOZAR-ZARZOI3-244450,thermo_id_20001336,54.7,21.8,Normal


In [40]:
# Filtrar dados para o termômetro de Araraquara (NORTE)
df_araraquara = df[df['regiao'].str.contains('NORTE', na=False)]

# Verificar se o DataFrame filtrado possui dados
if df_araraquara.empty:
    raise ValueError("Nenhum dado encontrado para a região ZAR no DataFrame fornecido.")

# 3a) Centro de Gravidade
def calcular_centro_gravidade(df, coluna_temperatura):
    # Verificar se a coluna possui valores não nulos
    if df[coluna_temperatura].isnull().sum() > 0:
        raise ValueError(f"A coluna {coluna_temperatura} possui valores nulos.")

    # Agrupar em blocos de 2 graus
    bins = np.arange(df[coluna_temperatura].min() - 1, df[coluna_temperatura].max() + 1, 2)
    df['bloco'] = pd.cut(df[coluna_temperatura], bins=bins)

    # Contar quantidade de dias em cada bloco
    blocos = df.groupby('bloco', observed=False).size().reset_index(name='quantidade')

    # Calcular X como o ponto médio do intervalo do bloco
    blocos['X'] = blocos['bloco'].apply(lambda x: (x.left + x.right) / 2 if not pd.isnull(x) else np.nan)

    # Garantir que os cálculos são feitos apenas com valores numéricos
    blocos['Y'] = blocos['quantidade'] / 2
    blocos['Area'] = 2 * blocos['quantidade']

    # Converter 'X' e 'Y' para numéricos antes de realizar as multiplicações
    blocos['X'] = pd.to_numeric(blocos['X'], errors='coerce')
    blocos['MX'] = blocos['Area'] * blocos['X']
    blocos['MY'] = blocos['Area'] * blocos['Y']

    # Centro de Gravidade
    somatorio_area = blocos['Area'].sum()
    cg_x = blocos['MX'].sum() / somatorio_area
    cg_y = blocos['MY'].sum() / somatorio_area

    # Faixa de Temperatura Neutra (FTN)
    ftn_centro = cg_x + 5
    ftn_inicio = cg_x
    ftn_fim = cg_x + 10

    return cg_x, cg_y, ftn_inicio, ftn_fim, blocos

# 3b) Min/Max
def calcular_min_max(df, coluna_temperatura):
    minimo = df[coluna_temperatura].min()
    maximo = df[coluna_temperatura].max()

    inicio_ftn = (minimo + maximo) / 2
    centro_ftn = inicio_ftn + 5
    fim_ftn = inicio_ftn + 10

    return inicio_ftn, centro_ftn, fim_ftn

inicio_ftn_max, centro_ftn_max, fim_ftn_max = calcular_min_max(df_araraquara, 'temperatura_maxima')
inicio_ftn_min, centro_ftn_min, fim_ftn_min = calcular_min_max(df_araraquara, 'temperatura_minima')

# 3c) Min/Max com 4Sigma
def calcular_min_max_4sigma(df, coluna_temperatura):
    # Verificar se a coluna possui valores não nulos
    if df[coluna_temperatura].isnull().sum() > 0:
        raise ValueError(f"A coluna {coluna_temperatura} possui valores nulos.")

    desvios = df[coluna_temperatura].std()
    media = df[coluna_temperatura].mean()

    limite_inferior = media - 4 * desvios
    limite_superior = media + 4 * desvios

    df_filtrado = df[(df[coluna_temperatura] >= limite_inferior) & (df[coluna_temperatura] <= limite_superior)]

    return calcular_min_max(df_filtrado, coluna_temperatura)

inicio_ftn_max_4sigma, centro_ftn_max_4sigma, fim_ftn_max_4sigma = calcular_min_max_4sigma(df_araraquara, 'temperatura_maxima')
inicio_ftn_min_4sigma, centro_ftn_min_4sigma, fim_ftn_min_4sigma = calcular_min_max_4sigma(df_araraquara, 'temperatura_minima')

# Cálculo de Centro de Gravidade
try:
    cg_x_max, cg_y_max, ftn_inicio_max, ftn_fim_max, blocos_max = calcular_centro_gravidade(df_araraquara, 'temperatura_maxima')
    cg_x_min, cg_y_min, ftn_inicio_min, ftn_fim_min, blocos_min = calcular_centro_gravidade(df_araraquara, 'temperatura_minima')
    print("Centro de Gravidade calculado com sucesso.")
except Exception as e:
    print(f"Erro ao calcular o Centro de Gravidade: {e}")
    cg_x_max, cg_y_max, ftn_inicio_max, ftn_fim_max = None, None, None, None
    cg_x_min, cg_y_min, ftn_inicio_min, ftn_fim_min = None, None, None, None

# Min/Max
try:
    inicio_ftn_max, centro_ftn_max, fim_ftn_max = calcular_min_max(df_araraquara, 'temperatura_maxima')
    inicio_ftn_min, centro_ftn_min, fim_ftn_min = calcular_min_max(df_araraquara, 'temperatura_minima')
    print("Min/Max calculado com sucesso.")
except Exception as e:
    print(f"Erro ao calcular Min/Max: {e}")
    inicio_ftn_max, centro_ftn_max, fim_ftn_max = None, None, None
    inicio_ftn_min, centro_ftn_min, fim_ftn_min = None, None, None

# Min/Max com 4Sigma
try:
    inicio_ftn_max_4sigma, centro_ftn_max_4sigma, fim_ftn_max_4sigma = calcular_min_max_4sigma(df_araraquara, 'temperatura_maxima')
    inicio_ftn_min_4sigma, centro_ftn_min_4sigma, fim_ftn_min_4sigma = calcular_min_max_4sigma(df_araraquara, 'temperatura_minima')
    print("Min/Max com 4Sigma calculado com sucesso.")
except Exception as e:
    print(f"Erro ao calcular Min/Max com 4Sigma: {e}")
    inicio_ftn_max_4sigma, centro_ftn_max_4sigma, fim_ftn_max_4sigma = None, None, None
    inicio_ftn_min_4sigma, centro_ftn_min_4sigma, fim_ftn_min_4sigma = None, None, None

# Resultados
resultados = {
    "Centro de Gravidade Max": (cg_x_max, cg_y_max, ftn_inicio_max, ftn_fim_max),
    "Centro de Gravidade Min": (cg_x_min, cg_y_min, ftn_inicio_min, ftn_fim_min),
    "Min/Max Max": (inicio_ftn_max, centro_ftn_max, fim_ftn_max),
    "Min/Max Min": (inicio_ftn_min, centro_ftn_min, fim_ftn_min),
    "4Sigma Max": (inicio_ftn_max_4sigma, centro_ftn_max_4sigma, fim_ftn_max_4sigma),
    "4Sigma Min": (inicio_ftn_min_4sigma, centro_ftn_min_4sigma, fim_ftn_min_4sigma)
}

# Exibir resultados
for key, value in resultados.items():
    print(f"{key}: {value}")

# Salvar dados filtrados
df_araraquara[['data', 'temperatura_maxima']].to_csv('C:/Users/Cristiano/OneDrive - Belago Technologies/Documentos/projeto_fnt/data_max_min/araraquara_temperatura_maxima.csv', index=False)
df_araraquara[['data', 'temperatura_minima']].to_csv('C:/Users/Cristiano/OneDrive - Belago Technologies/Documentos/projeto_fnt/data_max_min/araraquara_temperatura_minima.csv', index=False)
print("Dados salvos com sucesso.")


Centro de Gravidade calculado com sucesso.
Min/Max calculado com sucesso.
Min/Max com 4Sigma calculado com sucesso.
Centro de Gravidade Max: (np.float64(46.608549092734705), np.float64(1858.6883920246719), np.float64(46.608549092734705), np.float64(56.608549092734705))
Centro de Gravidade Min: (np.float64(17.06921806562792), np.float64(3579.608835355784), np.float64(17.06921806562792), np.float64(27.06921806562792))
Min/Max Max: (np.float64(585.0), np.float64(590.0), np.float64(595.0))
Min/Max Min: (np.float64(-175.0), np.float64(-170.0), np.float64(-165.0))
4Sigma Max: (np.float64(113.5), np.float64(118.5), np.float64(123.5))
4Sigma Min: (np.float64(-15.0), np.float64(-10.0), np.float64(-5.0))
Dados salvos com sucesso.


## Gráfico 01 Temperaturas Máximas e Mínimas por Ano em Araraquara

In [ ]:
df_araraquara['data'] = pd.to_datetime(df_araraquara['data'])

df_araraquara['ano'] = df_araraquara['data'].dt.year

df_ano = df_araraquara.groupby('ano')[['temperatura_maxima', 'temperatura_minima']].mean().reset_index()

fig = px.line(df_ano, x='ano', y=['temperatura_maxima', 'temperatura_minima'],
              labels={'temperatura_maxima': 'Temperatura Máxima (°C)', 'temperatura_minima': 'Temperatura Mínima (°C)', 'ano': 'Ano'},
              title='Temperaturas Máximas e Mínimas por Ano em Araraquara')

fig.show()


## Gráfico 02 Temperaturas Máximas e Mínimas por Ano em Araraquara

In [ ]:
df_araraquara['data'] = pd.to_datetime(df_araraquara['data'])
df_araraquara['ano'] = df_araraquara['data'].dt.year

df_ano = df_araraquara.groupby('ano')[['temperatura_maxima', 'temperatura_minima']].mean().reset_index()

x_data = df_ano['ano'].values
y_data_max = df_ano['temperatura_maxima'].values
y_data_min = df_ano['temperatura_minima'].values

colors = ['rgb(255, 99, 71)', 'rgb(70,130,180)'] 
mode_size = [10, 10]
line_size = [3, 3]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x_data, y=y_data_max, mode='lines',
                         name='Temperatura Máxima',
                         line=dict(color=colors[0], width=line_size[0]),
                         connectgaps=True))

fig.add_trace(go.Scatter(x=x_data, y=y_data_min, mode='lines',
                         name='Temperatura Mínima',
                         line=dict(color=colors[1], width=line_size[1]),
                         connectgaps=True))

fig.add_trace(go.Scatter(
    x=[x_data[0], x_data[-1]], 
    y=[y_data_max[0], y_data_max[-1]], 
    mode='markers',
    marker=dict(color=colors[0], size=mode_size[0]),
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=[x_data[0], x_data[-1]], 
    y=[y_data_min[0], y_data_min[-1]], 
    mode='markers',
    marker=dict(color=colors[1], size=mode_size[1]),
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=[x_data[0]], 
    y=[y_data_max[0]], 
    mode='text', 
    text=[f'{y_data_max[0]:.2f}°C'], 
    textposition='top right', 
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=[x_data[-1]], 
    y=[y_data_max[-1]], 
    mode='text', 
    text=[f'{y_data_max[-1]:.2f}°C'], 
    textposition='top right', 
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=[x_data[0]], 
    y=[y_data_min[0]], 
    mode='text', 
    text=[f'{y_data_min[0]:.2f}°C'], 
    textposition='top right', 
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=[x_data[-1]], 
    y=[y_data_min[-1]], 
    mode='text', 
    text=[f'{y_data_min[-1]:.2f}°C'], 
    textposition='top right', 
    showlegend=False
))

fig.update_layout(
    title='Temperaturas Máximas e Mínimas por Ano em Araraquara',
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(family='Arial', size=12, color='rgb(82, 82, 82)'),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=True,
        title='Temperatura (°C)',
    ),
    autosize=False,
    width=1100,
    height=600,
    margin=dict(autoexpand=False, l=100, r=20, t=110),
    plot_bgcolor='white'
)

fig.show()


## Centro de Gravidade Máximo e Mínimo por Ano

In [ ]:
if 'data' in df_araraquara.columns:
    df_araraquara['ano'] = pd.to_datetime(df_araraquara['data']).dt.year

if 'ano' not in df_araraquara.columns:
    raise ValueError("A coluna 'ano' não foi encontrada no dataframe.")

anos = df_araraquara['ano'].unique()  # Usando os anos do dataframe
centro_gravidade_max = [cg_x_max] * len(anos)  # Valores simulados para o centro de gravidade máximo
centro_gravidade_min = [cg_x_min] * len(anos)  # Valores simulados para o centro de gravidade mínimo

colors = ['rgb(255, 99, 71)', 'rgb(70,130,180)']  # Vermelho para Max, Azul para Min
mode_size = [10, 10]
line_size = [3, 3]

fig = go.Figure()

fig.add_trace(go.Scatter(x=anos, y=centro_gravidade_max, mode='lines+markers',
                         name='Centro de Gravidade Máximo',
                         line=dict(color=colors[0], width=line_size[0]),
                         marker=dict(size=mode_size[0]),
                         connectgaps=True))

fig.add_trace(go.Scatter(x=anos, y=centro_gravidade_min, mode='lines+markers',
                         name='Centro de Gravidade Mínimo',
                         line=dict(color=colors[1], width=line_size[1]),
                         marker=dict(size=mode_size[1]),
                         connectgaps=True))

fig.add_trace(go.Scatter(
    x=[anos[0]], 
    y=[centro_gravidade_max[0]], 
    mode='text', 
    text=[f'{centro_gravidade_max[0]:.2f}°C'], 
    textposition='top right', 
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=[anos[-1]], 
    y=[centro_gravidade_max[-1]], 
    mode='text', 
    text=[f'{centro_gravidade_max[-1]:.2f}°C'], 
    textposition='top right', 
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=[anos[0]], 
    y=[centro_gravidade_min[0]], 
    mode='text', 
    text=[f'{centro_gravidade_min[0]:.2f}°C'], 
    textposition='top right', 
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=[anos[-1]], 
    y=[centro_gravidade_min[-1]], 
    mode='text', 
    text=[f'{centro_gravidade_min[-1]:.2f}°C'], 
    textposition='top right', 
    showlegend=False
))

# Atualizando layout do gráfico
fig.update_layout(
    title='Centro de Gravidade Máximo e Mínimo por Ano',
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(family='Arial', size=12, color='rgb(82, 82, 82)'),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=True,
        title='Centro de Gravidade (°C)',
    ),
    autosize=False,
    width=1000,  # Largura aumentada
    height=600,  # Altura aumentada
    margin=dict(autoexpand=False, l=100, r=20, t=110),
    plot_bgcolor='white'
)

fig.show()

## gráfico 4Sigma Max e Min

In [45]:
if 'data' in df_araraquara.columns:
    # Se a coluna de data estiver em formato string ou datetime, extraímos o ano
    df_araraquara['ano'] = pd.to_datetime(df_araraquara['data']).dt.year

# Verificando se a coluna 'ano' foi criada corretamente
if 'ano' not in df_araraquara.columns:
    raise ValueError("A coluna 'ano' não foi encontrada no dataframe.")

# Definindo os dados para o 4Sigma (exemplo)
anos = df_araraquara['ano'].unique()  # Usando os anos do dataframe
sigma_max = [centro_ftn_max_4sigma] * len(anos)  # Valores simulados para o 4Sigma máximo
sigma_min = [centro_ftn_min_4sigma] * len(anos)  # Valores simulados para o 4Sigma mínimo

# Definindo as cores e o tamanho dos marcadores
colors = ['rgb(255, 69, 0)', 'rgb(30, 144, 255)']  # Laranja para Max, Azul para Min
mode_size = [10, 10]
line_size = [3, 3]

# Criando a figura
fig = go.Figure()

# Plotando a linha para o 4Sigma Máximo
fig.add_trace(go.Scatter(x=anos, y=sigma_max, mode='lines+markers',
                         name='4Sigma Máximo',
                         line=dict(color=colors[0], width=line_size[0]),
                         marker=dict(size=mode_size[0]),
                         connectgaps=True))

# Plotando a linha para o 4Sigma Mínimo
fig.add_trace(go.Scatter(x=anos, y=sigma_min, mode='lines+markers',
                         name='4Sigma Mínimo',
                         line=dict(color=colors[1], width=line_size[1]),
                         marker=dict(size=mode_size[1]),
                         connectgaps=True))

# Adicionando rótulos nas extremidades das linhas
fig.add_trace(go.Scatter(
    x=[anos[0]], 
    y=[sigma_max[0]], 
    mode='text', 
    text=[f'{sigma_max[0]:.2f}°C'], 
    textposition='top right', 
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=[anos[-1]], 
    y=[sigma_max[-1]], 
    mode='text', 
    text=[f'{sigma_max[-1]:.2f}°C'], 
    textposition='top right', 
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=[anos[0]], 
    y=[sigma_min[0]], 
    mode='text', 
    text=[f'{sigma_min[0]:.2f}°C'], 
    textposition='top right', 
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=[anos[-1]], 
    y=[sigma_min[-1]], 
    mode='text', 
    text=[f'{sigma_min[-1]:.2f}°C'], 
    textposition='top right', 
    showlegend=False
))

# Atualizando layout do gráfico
fig.update_layout(
    title='4Sigma Máximo e Mínimo por Ano',
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(family='Arial', size=12, color='rgb(82, 82, 82)'),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=True,
        title='4Sigma (°C)',
    ),
    autosize=False,
    width=1000,  # Largura aumentada
    height=600,  # Altura aumentada
    margin=dict(autoexpand=False, l=100, r=20, t=110),
    plot_bgcolor='white'
)

fig.show()

In [41]:
df_temp_max = pd.read_csv('C:/Users/Cristiano/OneDrive - Belago Technologies/Documentos/projeto_fnt/data_max_min/araraquara_temperatura_maxima.csv')
df_temp_max.head()

,data,temperatura_maxima
0,2023-11-08,54.2
1,2023-11-09,49.1
2,2023-11-10,56.1
3,2023-11-11,56.4
4,2023-11-12,57.6


In [42]:
df_temp_max = pd.read_csv('C:/Users/Cristiano/OneDrive - Belago Technologies/Documentos/projeto_fnt/data_max_min/araraquara_temperatura_minima.csv')
df_temp_max.head()

,data,temperatura_minima
0,2023-11-08,24.7
1,2023-11-09,19.1
2,2023-11-10,19.6
3,2023-11-11,18.8
4,2023-11-12,20.6
